http://blog.castman.net/%E6%95%99%E5%AD%B8/2016/12/20/python-data-science-tutorial-2.html
# 範例: PTT Beauty 板今日圖片下載器

In [1]:
import requests; import bs4; import matplotlib;import time;import re;import os;import json;import urllib.request

In [2]:
def get_web_page(url):
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

In [3]:
page = get_web_page('https://www.ptt.cc/bbs/Beauty/index.html')
if page:
	print(page)

<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		

<meta name="viewport" content="width=device-width, initial-scale=1">

<title>看板 Beauty 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/bbs-common.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/bbs-base.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/bbs-custom.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/pushstream.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/bbs-print.css" media="print">




	</head>
    <body>
		
<div id="topbar-container">
	<div id="topbar" class="bbs-content">
		<a id="logo" href="/bbs/">批踢踢實業坊</a>
		<span>&rsaquo;</span>
		<a class="board" href="/bbs/Beauty/index.html"><span class="board-label">看板 </span>Beauty</a>
		<a class="right small" href="/about.html">關於我們</a>
		<a class="right small" href="/contact.htm

網頁 = 由標籤 (tag) 所組成的階層式文件
你在瀏覽器看到的美觀網頁，主要由三個部分構成: HTML (網頁的骨架結構)、CSS (網頁的樣式) 與 JavaScript (在瀏覽器端執行，負責與使用者互動的程式功能)。對於網頁或爬蟲的初學者來說，最重要的觀念是了解：網頁就是由各式標籤 (tag) 所組成的階層式文件，要取得所需的網頁區塊資料，只要用 tag 與相關屬性去定位資料所在位置即可。

In [4]:
html_doc = """
<html>
  <head>
    <title>我是網頁標題</title>
    <style>
    .large {
      color:blue;
      text-align: center;
    }
    </style>
  </head>
  <body>
    <h1 class="large">我是變色且置中的抬頭</h1>
    <p id="p1">我是段落一</p>
    <p id="p2" style="">我是段落二</p>
    <div><a href='http://blog.castman.net' style="font-size:200%;">我是放大的超連結</a></div>
  </body>
</html>
"""

![](img/htmltree.png)

先創建一個 BeautifulSoup 物件，將網頁讀入

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')
print(soup)
# <html>
# <head>
# <title>我是網頁標題</title>
# <style>
# .large {
#   color:blue;
#   text-align: center;
# }
# </style>
# </head>
# <body>
# <h1 class="large" style="">我是變色且置中的抬頭</h1>
# <p id="p1">我是段落一</p>
# <p id="p2" style="">我是段落二</p>
# <div><a href="http://blog.castman.net" style="font-size:200%;">我是放大的超連結</a></div>
# </body>
# </html>


<html>
<head>
<title>我是網頁標題</title>
<style>
    .large {
      color:blue;
      text-align: center;
    }
    </style>
</head>
<body>
<h1 class="large">我是變色且置中的抬頭</h1>
<p id="p1">我是段落一</p>
<p id="p2" style="">我是段落二</p>
<div><a href="http://blog.castman.net" style="font-size:200%;">我是放大的超連結</a></div>
</body>
</html>



接著就可以用 find(), find_all() 搭配 tag 名稱及屬性去定位資料區塊

In [6]:
soup.find('p')            # 回傳第一個被 <p> </p> 所包圍的區塊
# <p id="p1">我是段落一</p>

<p id="p1">我是段落一</p>

In [7]:
soup.find('p', id='p2')   # 回傳第一個被 <p> </p> 所包圍的區塊且 id="p2"
# <p id="p2" style="">我是段落二</p>

<p id="p2" style="">我是段落二</p>

In [8]:
soup.find(id='p2')        # 回傳第一個 id="p2" 的區塊
# <p id="p2" style="">我是段落二</p>

<p id="p2" style="">我是段落二</p>

In [9]:
soup.find_all('p')        # 回傳所有被 <p> </p> 所包圍的區塊
# [<p id="p1">我是段落一</p>, <p id="p2" style="">我是段落二</p>]


[<p id="p1">我是段落一</p>, <p id="p2" style="">我是段落二</p>]

In [10]:
soup.find('h1', 'large')  # 找尋第一個 <h1> 區塊且 class="large"
# <h1 class="large" style="">我是變色且置中的抬頭</h1>

<h1 class="large">我是變色且置中的抬頭</h1>

find() 只回傳第一個找到的區塊，而 find_all() 會回傳一個 list, 包含所有符合條件的區塊。傳入的引數第一個通常是 tag 名稱，第二個引數若未指明屬性就代表 class 名稱，也可以直接使用 id 等屬性去定位區塊。定位到區塊後，可以取出其屬性與包含的字串值

In [11]:
paragraphs = soup.find_all('p')
for p in paragraphs:
    print(p['id'], p.text)
# p1 我是段落一
# p2 我是段落二

a = soup.find('a')
print(a['href'], a['style'], a.text)
# http://blog.castman.net font-size:200%; 我是放大的超連結

print(soup.find('h1')['class'])  # 因為 class 可以有多個值，故回傳 list
# ['large']

p1 我是段落一
p2 我是段落二
http://blog.castman.net font-size:200%; 我是放大的超連結
['large']


如果你要取得的屬性不存在，直接使用屬性名稱會出現錯誤訊息，因此若你不確定屬性是否存在，可以改用 get() 方法

In [12]:
#print(soup.find(id='p1')['style'])      # 會出現錯誤訊息, 因為 <p id="p1"> 沒有 style 屬性
print(soup.find(id='p1').get('style'))  # None

None


使用 Chrome 的開發者工具找到資料區塊的 tag 及屬性
假設你有一個想爬的網頁，要怎麼知道資料區塊所在的標籤及屬性呢？在此我們使用 Chrome 的開發者工具，以 Ptt Web 版 Beauty 板首頁為例，用 Chrome 連上 https://www.ptt.cc/bbs/Beauty/index.html , 接著按下 F12 或從選單啟動開發者工具

![](img/161222-python-ds-tutorial-3-3.png)
下方會跑出開發者工具的操作區，點選左上角的箭頭按鈕後，再點擊網頁上你想要定位的資料區塊，該區塊的 HTML 碼就會顯示在下方。當然你也可以直接檢視網頁原始碼或檢視上一篇教學中用 get_web_page() 所取得的網頁文件，但善用開發者工具可以加速你的搜尋。
![](img/161222-python-ds-tutorial-3-4.png)
![](img/161222-python-ds-tutorial-3-5.png)

PTT Beauty 板範例實戰
檢視網頁原始碼後我們知道，網頁上的每一篇貼文都是由 <div class=”r-ent”> 的區塊包圍起來，裡面分別由 <div class=”nrec”> 區塊顯示推文數，<div class=”title”> 區塊及 <a> 區塊顯示文章連結及文章標題，<div class=”date”> 區塊顯示發文日期
![](img/161220-python-ds-tutorial-2-2.png)

因此，若已經取得網頁文件，我們可以用 find_all() 找出所有<div class=”r-ent”> 區塊，並逐一巡訪，取得資料：

In [13]:
def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'html.parser')

    # 取得上一頁的連結
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']

    articles = []  # 儲存取得的文章資料
    divs = soup.find_all('div', 'r-ent')
    for d in divs:
        if d.find('div', 'date').string.strip() == date:  # 發文日期正確
            # 取得推文數
            push_count = 0
            if d.find('div', 'nrec').string:
                try:
                    push_count = int(d.find('div', 'nrec').string)  # 轉換字串為數字
                except ValueError:  # 若轉換失敗，不做任何事，push_count 保持為 0
                    pass

            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').string
                articles.append({
                    'title': title,
                    'href': href,
                    'push_count': push_count
                })
    return articles, prev_url

使用 get_articles() 及上一篇教學的 get_web_page()，取得今日文章資訊

In [14]:
page = get_web_page('https://www.ptt.cc/bbs/Beauty/index.html')
if page:
    mm = time.strftime("%m").lstrip('0')  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
    mm = mm if (len(mm) > 1) else ' ' + mm
    dd = time.strftime("%d")  
    date = mm + '/' + dd
    current_articles = get_articles(page, date)
    for post in current_articles:
        print(post)

[]
/bbs/Beauty/index2479.html


https://eastlakeside.gitbooks.io/interpy-zh/content/ternary_operators/ternary_operators.html
三元运算符
三元运算符通常在Python里被称为条件表达式，这些表达式基于真(true)/假(false)的条件判断，在Python 2.4以上才有了三元操作。
下面是一个伪代码和例子：
伪代码:

#如果条件为真，返回真 否则返回假
condition_is_true if condition else condition_is_false
例子:

is_fat = True
state = "fat" if is_fat else "not fat"

In [15]:
def parse(dom):
    soup = BeautifulSoup(dom, 'html.parser')
    links = soup.find(id='main-content').find_all('a')
    img_urls = []
    for link in links:
        if re.match(r'^https?://(i.)?(m.)?imgur.com', link['href']):
            img_urls.append(link['href'])
    return img_urls

透過正規表示式 (Regular Expression) 指定字串的格式。例如能辨識出以上全部格式的正規表示式為:

'^https?://(i.)?(m.)?imgur.com'
”^” 表示字串開頭，字元緊接著 “?” 表示該字元可出現 0 或 1 次，所以 “^https?” 表示的是 “http” (s 出現 0 次) 或 “https” (s 出現 1 次) 開頭的字串，同理 (i.)? 表示 “i.” 可以出現 0 或 1 次。我們用 re.match() 判斷字串是否符合所定義的正規表示式：

In [16]:
def save(img_urls, title):
    if img_urls:
        try:
            dname = title.strip()  # 用 strip() 去除字串前後的空白
            os.makedirs(dname)
            for img_url in img_urls:
                if img_url.split('//')[1].startswith('m.'):
                    img_url = img_url.replace('//m.', '//i.')
                if not img_url.split('//')[1].startswith('i.'):
                    img_url = img_url.split('//')[0] + '//i.' + img_url.split('//')[1]
                if not img_url.endswith('.jpg'):
                    img_url += '.jpg'
                fname = img_url.split('/')[-1]
                urllib.request.urlretrieve(img_url, os.path.join(dname, fname))
        except Exception as e:
            print(e)

主程式:
    1.連線到網站，取得該文章網頁 (get_web_page())
    2.找到文章內的圖片網址們 (parse())
    3.在本機新增以文章標題為名的資料夾，將圖片存到本機 (save())
    4.紀錄圖片數量；繼續巡訪下一篇文章直到沒有文章為止

In [17]:
PTT_URL = 'https://www.ptt.cc'

In [18]:
if __name__ == '__main__':
    current_page = get_web_page(PTT_URL + '/bbs/Beauty/index.html')
    if current_page:
        articles = []  # 全部的今日文章
        date = time.strftime("%m/%d").lstrip('0')  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
        current_articles, prev_url = get_articles(current_page, date)  # 目前頁面的今日文章
        while current_articles:  # 若目前頁面有今日文章則加入 articles，並回到上一頁繼續尋找是否有今日文章
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page, date)

        # 已取得文章列表，開始進入各文章讀圖
        for article in articles:
            print('Processing', article)
            page = get_web_page(PTT_URL + article['href'])
            if page:
                img_urls = parse(page)
                save(img_urls, article['title'])
                article['num_image'] = len(img_urls)

        # 儲存文章資訊
        with open('data.json', 'w', encoding='utf-8') as f:
            json.dump(articles, f, indent=2, sort_keys=True, ensure_ascii=False)


Processing {'title': '[正妹] 甜美可愛日本人', 'href': '/bbs/Beauty/M.1526056001.A.B71.html', 'push_count': 5}
